<a href="https://colab.research.google.com/github/Avatar-Mail/FineTuning-avatar-mail/blob/main/%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D_%EB%AA%A8%EB%8D%B8_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 램 확장하기

## coqui 설치

In [ ]:
!pip uninstall -y numpy tensorflow tensorboard TTS
!pip install numpy==1.23.0 tensorflow tensorboard TTS

!git clone https://github.com/coqui-ai/TTS


Found existing installation: numpy 1.23.0
Uninstalling numpy-1.23.0:
  Successfully uninstalled numpy-1.23.0
Found existing installation: tensorflow 2.11.1
Uninstalling tensorflow-2.11.1:
  Successfully uninstalled tensorflow-2.11.1
Found existing installation: tensorboard 2.11.2
Uninstalling tensorboard-2.11.2:
  Successfully uninstalled tensorboard-2.11.2
Found existing installation: TTS 0.14.3
Uninstalling TTS-0.14.3:
  Successfully uninstalled TTS-0.14.3
  Using cached numpy-1.23.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.2 kB)
  Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached tensorboard-2.17.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached TTS-0.22.0-cp310-cp310-manylinux1_x86_64.whl.metadata (21 kB)
  Using cached keras-3.4.1-py3-none-any.whl.metadata (5.8 kB)
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requir

Cloning into 'TTS'...
remote: Enumerating objects: 32844, done.
remote: Total 32844 (delta 0), reused 0 (delta 0), pack-reused 32844
Receiving objects: 100% (32844/32844), 166.21 MiB | 14.74 MiB/s, done.
Resolving deltas: 100% (23820/23820), done.


In [ ]:
import os
os.chdir('/content/TTS')
!pip install -e .[all]

Obtaining file:///content/TTS
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached Cython-3.0.11-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.2 kB)
  Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached trainer-0.0.36-py3-none-any.whl.metadata (8.1 kB)
  Using cached hangul_romanize-0.1.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached encodec-0.1.1.tar.gz (3.7 MB)
  Preparing metadata (setup.py) ... done
  Using cached Unidecode-1.3.8-py3-none-any.whl.metadata (13 kB)
  Using cached numba-0.60.0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.4/32.4 MB 46.4 MB/s eta 0:00:0

## 초기 설정 세팅

In [ ]:
import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager

# Logging parameters
RUN_NAME = "GPT_XTTS_v2.0_JISUN_FT"
PROJECT_NAME = "XTTS_trainer"
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None

# Set here the path that the checkpoints will be saved. Default: ./run/training/
OUT_PATH = "/content/run/training"

# Training Parameters
OPTIMIZER_WD_ONLY_ON_WEIGHTS = True  # for multi-gpu training please make it False
START_WITH_EVAL = True  # if True it will star with evaluation
BATCH_SIZE = 3  # set here the batch size
GRAD_ACUMM_STEPS = 84  # set here the grad accumulation steps
# Note: we recommend that BATCH_SIZE * GRAD_ACUMM_STEPS need to be at least 252 for more efficient training. You can increase/decrease BATCH_SIZE but then set GRAD_ACUMM_STEPS accordingly.

## 구글 드라이브에서 데이터셋 로드

In [ ]:
# 구글 드라이브에서 데이터셋 불러오기
from google.colab import drive
drive.mount('/content/drive')

zip_file_path = '/content/drive/MyDrive/jisun-tts-dataset/jisun.zip'
extract_path = '/'

# 파일 압축 해제
import zipfile
import os

os.makedirs(extract_path, exist_ok=True)

# 압축 해제
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


Mounted at /content/drive


In [ ]:
zip_file_path = '/content/drive/MyDrive/jisun-model.zip'
extract_path = '/'
os.makedirs(extract_path, exist_ok=True)

# 압축 해제
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

## 데이터셋 세팅후 모델 파인튜닝 설정

In [ ]:

# Define here the dataset that you want to use for the fine-tuning on.
config_dataset = BaseDatasetConfig(
    formatter="ljspeech",
    dataset_name="jisun",
    path="/content/dataset",
    meta_file_train="/content/dataset/metadata.txt",
    language="ko",
)

# Add here the configs of the datasets
DATASETS_CONFIG_LIST = [config_dataset]

# Define the path where XTTS v2.0.1 files will be downloaded
CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)


# DVAE files
DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

# Set the path to the downloaded files
DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

# download DVAE files if needed
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True)


# Download XTTS v2.0 checkpoint if needed
TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"

# XTTS transfer learning parameters: You we need to provide the paths of XTTS model checkpoint that you want to do the fine tuning.
TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))  # vocab.json file
XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))  # model.pth file

# download XTTS v2.0 files if needed
if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True
    )


# Training sentences generations
SPEAKER_REFERENCE = [
    "/content/dataset/wavs/audio1.wav"  # speaker reference to be used in training test sentences
]
LANGUAGE = config_dataset.language


 > Downloading DVAE files!


  0%|          | 0.00/1.07k [00:00<?, ?iB/s]
100%|██████████| 1.07k/1.07k [00:00<00:00, 1.33kiB/s]

  1%|▏         | 2.89M/211M [00:00<00:07, 28.9MiB/s]
  6%|▌         | 12.2M/211M [00:00<00:02, 66.8MiB/s]
 10%|▉         | 20.1M/211M [00:00<00:02, 72.1MiB/s]
 14%|█▎        | 28.4M/211M [00:00<00:02, 76.2MiB/s]
 17%|█▋        | 36.1M/211M [00:00<00:02, 72.4MiB/s]
 21%|██        | 43.3M/211M [00:00<00:02, 72.2MiB/s]
 24%|██▍       | 51.1M/211M [00:00<00:02, 73.9MiB/s]
 28%|██▊       | 58.5M/211M [00:00<00:02, 68.0MiB/s]
 31%|███▏      | 66.0M/211M [00:00<00:02, 69.9MiB/s]
 35%|███▌      | 73.9M/211M [00:01<00:01, 72.7MiB/s]
 39%|███▉      | 82.3M/211M [00:01<00:01, 75.9MiB/s]
 43%|████▎     | 89.9M/211M [00:01<00:01, 75.9MiB/s]
 46%|████▋     | 97.6M/211M [00:01<00:01, 74.9MiB/s]
 50%|████▉     | 105M/211M [00:01<00:01, 74.9MiB/s] 
 53%|█████▎    | 113M/211M [00:01<00:01, 70.8MiB/s]
 57%|█████▋    | 120M/211M [00:01<00:01, 72.1MiB/s]
 61%|██████    | 128M/211M [00:01<00:01, 74.9MiB/s]
 6

 > Downloading XTTS v2.0 files!


100%|██████████| 211M/211M [00:03<00:00, 62.1MiB/s]

100%|██████████| 361k/361k [00:00<00:00, 434kiB/s]

  0%|          | 4.40M/1.87G [00:00<00:42, 44.0MiB/s]
  1%|          | 10.6M/1.87G [00:00<00:34, 53.6MiB/s]
  1%|          | 18.5M/1.87G [00:00<00:28, 65.4MiB/s]
  1%|▏         | 26.2M/1.87G [00:00<00:26, 70.0MiB/s]
  2%|▏         | 34.3M/1.87G [00:00<00:24, 73.9MiB/s]
  2%|▏         | 41.7M/1.87G [00:00<00:25, 72.0MiB/s]
  3%|▎         | 48.9M/1.87G [00:00<00:25, 71.2MiB/s]
  3%|▎         | 56.3M/1.87G [00:00<00:25, 72.0MiB/s]
  3%|▎         | 63.5M/1.87G [00:00<00:26, 67.1MiB/s]
  4%|▍         | 71.6M/1.87G [00:01<00:25, 71.1MiB/s]
  4%|▍         | 78.8M/1.87G [00:01<00:26, 67.4MiB/s]
  5%|▍         | 85.6M/1.87G [00:01<00:26, 67.4MiB/s]
  5%|▌         | 93.6M/1.87G [00:01<00:24, 71.1MiB/s]
  5%|▌         | 101M/1.87G [00:01<00:24, 72.0MiB/s] 
  6%|▌         | 108M/1.87G [00:01<00:26, 67.6MiB/s]
  6%|▌         | 115M/1.87G [00:01<00:28, 61.3MiB/s]
  7%|▋         | 123M/1.87G [00:0

## 모델 파인튜닝 시작

In [ ]:
def main():
    # init args and config
    model_args = GPTArgs(
        max_conditioning_length=132300,  # 6 secs
        min_conditioning_length=66150,  # 3 secs
        debug_loading_failures=False,
        max_wav_length=255995,  # ~11.6 seconds
        max_text_length=200,
        mel_norm_file=MEL_NORM_FILE,
        dvae_checkpoint=DVAE_CHECKPOINT,
        xtts_checkpoint=XTTS_CHECKPOINT,  # checkpoint path of the model that you want to fine-tune
        tokenizer_file=TOKENIZER_FILE,
        gpt_num_audio_tokens=1026,
        gpt_start_audio_token=1024,
        gpt_stop_audio_token=1025,
        gpt_use_masking_gt_prompt_approach=True,
        gpt_use_perceiver_resampler=True,
    )
    # define audio config
    audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=24000)
    # training parameters config
    config = GPTTrainerConfig(
        output_path=OUT_PATH,
        model_args=model_args,
        run_name=RUN_NAME,
        project_name=PROJECT_NAME,
        run_description="""
            GPT XTTS training
            """,
        dashboard_logger=DASHBOARD_LOGGER,
        logger_uri=LOGGER_URI,
        audio=audio_config,
        batch_size=BATCH_SIZE,
        batch_group_size=48,
        eval_batch_size=BATCH_SIZE,
        num_loader_workers=8,
        eval_split_max_size=256,
        print_step=50,
        plot_step=100,
        log_model_step=1000,
        save_step=10000,
        save_n_checkpoints=1,
        save_checkpoints=True,
        # target_loss="loss",
        print_eval=False,
        # Optimizer values like tortoise, pytorch implementation with modifications to not apply WD to non-weight parameters.
        optimizer="AdamW",
        optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
        optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
        lr=5e-06,  # learning rate
        lr_scheduler="StepLR",
        # it was adjusted accordly for the new step scheme
        lr_scheduler_params={"step_size": 50, "gamma": 0.5, "last_epoch": -1},
        epochs=100,
        test_sentences=[
            {
                "text": "나에게는 그들보다 이 점등인이 더 나은 사람이야. 적어도 점등인은 그들과는 달리, 남을 위해 일하기 때문이야. 너는 나에게 이 세상에 단 하나뿐인 존재가 되는 거고, 나도 너에게 세상에 하나뿐인 존재가 되는 거야.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "넌 네가 길들인 것에 영원히 책임이 있어. 누군가에게 길들여진다는 것은 눈물을 흘릴 일이 생긴다는 뜻일지도 몰라. 네 장미꽃이 소중한 이유는 그 꽃을 위해 네가 애쓴 시간 때문이야. 다른 사람에게는 열어주지 않는 문을 당신에게만 열어주는 사람이 있다면 그 사람은 당신의 진정한 친구이다.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "세상에서 가장 어려운 일은 사람이 사람의 마음을 얻는 일이야. 내가 좋아하는 사람이 나를 좋아해 주는 건 기적이야.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "질문을 하지 않으면 세상 일을 어떻게 알겠어요?",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "세상엔 재미있는 일이 참 많아요. 우리가 모든 걸 다 안다면 사는 재미가 반으로 줄어들 거예요.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "또 다른 걱정거리들이 생길 거예요. 항상 골치 아픈 일들은 새롭게 일어나니까요. 한 가지가 해결되면 또 다른 문제가 이어지죠. 나이를 먹으니 생각할 것도, 결정해야 할 일도 많아져요. 뭐가 옳은지 곰곰이 생각하고 결정하느라 늘 바빠요. 어른이 된다는 건 쉬운 일이 아니에요.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "이 세상에서 무언가를 얻거나 이루려면 반드시 그만한 대가를 치러야 한다. 야망을 품는 것은 가치 있는 일이지만 합당한 노력과 절제와 불안과 좌절 없이 얻어지지는 않는 법이다.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "앤은 발 앞에 높인 길이 아무리 좁더라도 그 길을 따라 잔잔한 행복의 꽃이 피어난다는 걸 알고 있었다. 정직한 일과 훌륭한 포부와 마음 맞는 친구가 있다는 기쁨은 온전히 앤의 것이었다.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "아무리 황량하고 따분해도 다른 아름다운 곳보다 고향에서 살고 싶어 하는 게 사람이에요. 세상에 집보다 좋은 곳은 없거든요.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "바보들은 심장이 있어도 그걸로 무엇을 해야 하는지 몰라요.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "살아 있는 존재라면 누구나 위험 앞에서 두려움을 느껴. 진정한 용기란 두려움에도 불구하고 위험에 맞서는 것인데, 너는 이미 그런 용기를 충분히 갖고 있어.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "너는 작가가 되지 않아도, 배우가 되지 않아도, 그저 너이기에 사랑스럽고 완전한 존재란다. 다른 무엇이 되려고 애쓰지 않아도 좋아. 너는 그저 너, 너다운 너이기만 하면 된단다.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "소설을 쓰며 사는 삶보다 소설처럼 살아가는 삶이 훨씬 더 재미있을 거예요.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "저는 매 순간 제가 행복하다는 사실을 온전히 느껴요. 아무리 속상한 일이 생겨도 그 사실을 잊지 않을 거예요.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "저는 인간에게 가장 필요한 자질은 상상력이라고 생각합니다. 상상력이 있어야 타인을 이해할 수 있고, 그래야 친절할 수도, 남을 이해할 수도, 또 동정할 수도 있으니까요.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "자신에게 찾아오는 기회를 붙잡을 의지만 있다면 세상은 행복으로 가득 차 있고, 가볼 곳도 많아요. 비법은 바로 유연한 마음이에요.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "정말 소중한 것은 커다란 기쁨이 아니에요. 사소한 곳에서 얻는 기쁨이 더 중요해요.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "큰 시련이 닥쳤을 때만 인격이 필요한 게 아니에요. 위기에 대처하거나, 치명적인 비극에 맞서는 건 누구나 할 수 있지만, 그날그날의 사소한 불운들을 웃음으로 넘기는 일은 '정신력'이 없다면 불가능한 일이에요. 제가 키워나가야 할 게 바로 이런 종류의 인격이에요.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "눈이 보이지 않으면 눈이 보이는 코끼리와 살을 맞대고 걸으면 되고, 다리가 불편하면 다리가 튼튼한 코끼리에게 기대서 걸으면 돼. 같이 있으면 그런 건 큰 문제가 아니야.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "누구든 너를 좋아하게 되면, 네가 누구인지 알아볼 수 있어. 아마 처음에는 호기심으로 너를 관찰하겠지. 하지만 점 너를 좋아하게 되어서 너를 눈여겨보게 되고, 네가 가까이 있을 때는 어떤 냄새가 나는지 알게 될 거고, 네가 걸을 때는 어떤 소리가 나는지에도 귀 기울이게 될 거야. 그게 바로 너야.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
        ],
    )

    # init the model from config
    model = GPTTrainer.init_from_config(config)

    # load training samples
    train_samples, eval_samples = load_tts_samples(
        DATASETS_CONFIG_LIST,
        eval_split=True,
        eval_split_max_size=config.eval_split_max_size,
        eval_split_size=config.eval_split_size,
    )

    # init the trainer and 🚀
    trainer = Trainer(
        TrainerArgs(
            restore_path="/content/best_model.pth",  # xtts checkpoint is restored via xtts_checkpoint key so no need of restore it using Trainer restore_path parameter
            skip_train_epoch=False,
            start_with_eval=START_WITH_EVAL,
            grad_accum_steps=GRAD_ACUMM_STEPS,
        ),
        config,
        output_path=OUT_PATH,
        model=model,
        train_samples=train_samples,
        eval_samples=eval_samples,
    )
    trainer.fit()


if __name__ == "__main__":
    main()


 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 8
 | > Num. of Torch Threads: 1
 | > Torch seed: 1
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False


>> DVAE weights restored from: /content/run/training/XTTS_v2.0_original_model_files/dvae.pth
 | > Found 232 files in /content/dataset


 > Start Tensorboard: tensorboard --logdir=/content/run/training/GPT_XTTS_v2.0_JISUN_FT-August-07-2024_02+14PM-0000000
 > Restoring from best_model.pth ...
 > Restoring Model...
 > Restoring Optimizer...
 > Model restored from step 4543

 > Model has 518442047 parameters

 > EPOCH: 0/100
 --> /content/run/training/GPT_XTTS_v2.0_JISUN_FT-August-07-2024_02+14PM-0000000


 > Filtering invalid eval samples!!



 > EVALUATION 



...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0653390884399414 (+0.0007157325744628906)
     | > avg_loss_text_ce: 0.021996865049004555 (-1.1567026376724243e-06)
     | > avg_loss_mel_ce: 3.209190845489502 (+0.0007710456848144531)
     | > avg_loss: 3.2311878204345703 (+0.0007700920104980469)


 > EPOCH: 38/100
 --> /content/run/training/GPT_XTTS_v2.0_JISUN_FT-August-07-2024_02+14PM-0000000

 > TRAINING (2024-08-07 16:07:24) 

   --> TIME: 2024-08-07 16:07:30 -- STEP: 7/77 -- GLOBAL_STEP: 7400
     | > loss_text_ce: 0.02439936436712742  (0.025155167494501387)
     | > loss_mel_ce: 3.1647491455078125  (3.1412192412785123)
     | > loss: 0.0379660539329052  (0.037694934223379405)
     | > current_lr: 5e-06 
     | > step_time: 0.267  (0.29758950642177034)
     | > loader_time: 0.0066  (0.007863623755318778)


   --> TIME: 2024-08-07 16:07:55 -- STEP: 57/77 -- GLOBAL_STEP: 7450
     | > loss_text_ce: 0.02273407205939293  (0.02457534708082676)
     | > loss_mel_ce: 3.189791679382324 

 | > Synthesizing test sentences.
[!] Warning: The text length exceeds the character limit of 95 for language 'ko', this might cause truncated audio.
[!] Warning: The text length exceeds the character limit of 95 for language 'ko', this might cause truncated audio.
[!] Warning: The text length exceeds the character limit of 95 for language 'ko', this might cause truncated audio.
[!] Warning: The text length exceeds the character limit of 95 for language 'ko', this might cause truncated audio.
[!] Warning: The text length exceeds the character limit of 95 for language 'ko', this might cause truncated audio.
[!] Warning: The text length exceeds the character limit of 95 for language 'ko', this might cause truncated audio.
[!] Warning: The text length exceeds the character limit of 95 for language 'ko', this might cause truncated audio.
[!] Warning: The text length exceeds the character limit of 95 for language 'ko', this might cause truncated audio.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0646200180053711 (-0.0007190704345703125)
     | > avg_loss_text_ce: 0.02199520543217659 (-1.6596168279647827e-06)
     | > avg_loss_mel_ce: 3.2101237773895264 (+0.0009329319000244141)
     | > avg_loss: 3.232119083404541 (+0.0009312629699707031)


 > EPOCH: 39/100
 --> /content/run/training/GPT_XTTS_v2.0_JISUN_FT-August-07-2024_02+14PM-0000000

 > TRAINING (2024-08-07 16:09:44) 

   --> TIME: 2024-08-07 16:10:01 -- STEP: 30/77 -- GLOBAL_STEP: 7500
     | > loss_text_ce: 0.024236973375082016  (0.024343055052061875)
     | > loss_mel_ce: 3.12406063079834  (3.117901849746704)
     | > loss: 0.03747973218560219  (0.03740767848988374)
     | > current_lr: 5e-06 
     | > step_time: 0.3078  (0.2977761427561443)
     | > loader_time: 0.0074  (0.007744510968526204)


 > EVALUATION 



 | > Synthesizing test sentences.
[!] Warning: The text length exceeds the character limit of 95 for language 'ko', this might cause truncated audio.
[!] Warning: The text length exceeds the character limit of 95 for language 'ko', this might cause truncated audio.
[!] Warning: The text length exceeds the character limit of 95 for language 'ko', this might cause truncated audio.
[!] Warning: The text length exceeds the character limit of 95 for language 'ko', this might cause truncated audio.
[!] Warning: The text length exceeds the character limit of 95 for language 'ko', this might cause truncated audio.
[!] Warning: The text length exceeds the character limit of 95 for language 'ko', this might cause truncated audio.
[!] Warning: The text length exceeds the character limit of 95 for language 'ko', this might cause truncated audio.
[!] Warning: The text length exceeds the character limit of 95 for language 'ko', this might cause truncated audio.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.06616091728210449 (+0.0015408992767333984)
     | > avg_loss_text_ce: 0.021991515532135963 (-3.689900040626526e-06)
     | > avg_loss_mel_ce: 3.2111411094665527 (+0.0010173320770263672)
     | > avg_loss: 3.2331326007843018 (+0.0010135173797607422)


 > EPOCH: 40/100
 --> /content/run/training/GPT_XTTS_v2.0_JISUN_FT-August-07-2024_02+14PM-0000000

 > TRAINING (2024-08-07 16:11:52) 

   --> TIME: 2024-08-07 16:11:56 -- STEP: 3/77 -- GLOBAL_STEP: 7550
     | > loss_text_ce: 0.02542407251894474  (0.02366677050789197)
     | > loss_mel_ce: 3.254514455795288  (3.0790541966756186)
     | > loss: 0.039046887308359146  (0.03693715607126554)
     | > current_lr: 5e-06 
     | > step_time: 0.2681  (0.28771106402079266)
     | > loader_time: 0.0107  (0.008684873580932617)

 > Keyboard interrupt detected.
 > Saving model before exiting...

 > CHECKPOINT : /content/run/training/GPT_XTTS_v2.0_JISUN_FT-August-07-2024_02+14PM-0000000/checkpoint_7553.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!mkdir /content/drive/MyDrive/jisun-tts-model

!mv /content/run/training/XTTS_v2.0_original_model_files/vocab.json /content/run/training/GPT_XTTS_v2.0_JISUN_FT-August-07-2024_02+14PM-0000000/vocab.json

!zip -r /content/jisun-model.zip /content/run/training/GPT_XTTS_v2.0_JISUN_FT-August-07-2024_02+14PM-0000000/best_model.pth \
/content/run/training/GPT_XTTS_v2.0_JISUN_FT-August-07-2024_02+14PM-0000000/config.json \
/content/run/training/GPT_XTTS_v2.0_JISUN_FT-August-07-2024_02+14PM-0000000/vocab.json
!cp -a /content/jisun-model.zip /content/drive/MyDrive/jisun-tts-model

mkdir: cannot create directory ‘/content/drive/MyDrive/jisun-tts-model’: File exists
mv: cannot stat '/content/run/training/XTTS_v2.0_original_model_files/vocab.json': No such file or directory
  adding: content/run/training/GPT_XTTS_v2.0_JISUN_FT-August-07-2024_02+14PM-0000000/best_model.pth (deflated 9%)
  adding: content/run/training/GPT_XTTS_v2.0_JISUN_FT-August-07-2024_02+14PM-0000000/config.json (deflated 76%)
  adding: content/run/training/GPT_XTTS_v2.0_JISUN_FT-August-07-2024_02+14PM-0000000/vocab.json (deflated 81%)


In [ ]:
!cp -a /content/jisun.zip /content/drive/MyDrive/jisun-tts-model

cp: cannot stat '/content/jisun.zip': No such file or directory
